In [0]:
filename = dbutils.widgets.get('filename')
print(filename)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-6323258672200346>, line 1
----> 1 filename = dbutils.widgets.get('filename')
      2 print(filename)

File /databricks/python_shell/lib/dbruntime/WidgetHandlerImpl.py:82, in WidgetsHandlerImpl.get(self, name)
     42 def get(self, name: str) -> str:
     43     """ Returns the current value of a widget with the given name.
     44 
     45     :param name: Name of the argument to be accessed
   (...)
     80         ```
     81     """
---> 82     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py:1362, in JavaMember.__call__(self, *args)
   1356 command = proto.CALL_COMMAND_NAME +\
   1357     self.command_header +\
   1358     args_command +\
   1359     proto.END_COMMAND_PART
   1361 answer = self.gateway_

In [0]:
alreadyMounted = False
dbutils.fs.mounts()

for x in dbutils.fs.mounts():
    if x.mountPoint == "/mnt/sales":
        alreadyMounted = True
        break
else:
        alreadyMounted = False
print(alreadyMounted)

if not alreadyMounted:
    dbutils.fs.mount(
        source='wasbs://sales@jagdishsa.blob.core.windows.net',
        mount_point='/mnt/sales',
        extra_configs = {'fs.azure.account.key.jagdishsa.blob.core.windows.net':
            'G77CoJStEtz7oli5oJDtwu+qkxq0/FeV5WiFOoNCImzSTb5gLl621FReoxaj5r+Dlff4oy/7Pkym+AStcmfC8Q=='})
    alreadyMounted = True
    print("Mounting Done Successfully")
else:
    print("Mounting Already Done")

True
Mounting Already Done


In [0]:
dbutils.fs.unmount("/mnt/sales")

/mnt/sales has been unmounted.


True

In [0]:
orders_df = spark.read.csv('/mnt/sales/landing/{}'.format(filename),inferSchema=True,header=True)

In [0]:
display(orders_df)

order_id,order_date,customer_id,order_status
1111111,25/07/2013 00:00:00,11599,CLOSED
2222222,25/07/2013 00:00:00,256,PENDING_PAYMENT
3333333,25/07/2013 00:00:00,12111,COMPLETE
4444444,25/07/2013 00:00:00,8827,CLOSED


In [0]:
errorFlg = False

orders_count = orders_df.count()
print(orders_count)

distinctOrders = orders_df.select("order_id").distinct().count()
print(distinctOrders)

if distinctOrders != orders_count:
    errorFlg = True

if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg":"true","errorMsg":"Repeated OrderID Found"}')

orders_df.createOrReplaceTempView("orders")

In [0]:
dbServer = 'capstonesales'
dbName = 'capstoneDb'
dbPort = '1433'
dbUser = 'mysql'
dbPassword = 'SQL-Pass'
DataBricksScope = 'salesCapstoneScope'

In [0]:
connectionurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer,dbPort,dbName,dbUser)
dbPassword = dbutils.secrets.get(scope = DataBricksScope, key = 'SQL-Pass')
connectionproperties = {
    'password':dbPassword,
    'user':dbUser,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
    }

In [0]:
ValidStatusDf = spark.read.jdbc(url=connectionurl, table = 'dbo.valid_orders_status', properties = connectionproperties)
display(ValidStatusDf)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELLED
PENDING_PAYMENT


In [0]:
ValidStatusDf.createOrReplaceTempView("valid_status")

In [0]:
invalidRowsDf = spark.sql("select * from orders where order_status not in (select * from valid_status)")

In [0]:
display(invalidRowsDf)

order_id,order_date,customer_id,order_status


In [0]:
if invalidRowsDf.count() > 0:
    errorFlg = True

if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg":"true","errorMsg":"Invalid Order Status Found"}')
else: 
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/staging')
    ###dbutils.notebook.exit('{"errorFlg":"false","errorMsg":"All Good"}')### Removing as we dont want to Exit

In [0]:
orderItemDf = spark.read.csv('')